## The main purpose of this file is to extract useful features from the data, for regression based tasks !! 

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import kurtosis
from scipy.fft import rfft, rfftfreq

# 🧩 Feature Extraction - description of each feature

This preprocessing step converts each raw high-frequency cut file into a compact set of **10 physically meaningful features**.  
The motivation is to summarize the rich sensor data (forces, vibrations, and acoustic emissions) into quantities that capture **tool load, stability, and degradation behavior**.

---

## ⚙️ Feature Overview and Justification

### **1. Force RMS Magnitude (`F_rms`)**
\[
F_{rms} = \sqrt{mean(F_x^2 + F_y^2 + F_z^2)}
\]
- Represents the overall mechanical load on the cutter.  
- As the tool wears, cutting forces typically increase due to friction and reduced sharpness.  
- A stable but rising `F_rms` trend is a strong indicator of progressive wear.

---

### **2. Vibration RMS Magnitude (`V_rms`)**
\[
V_{rms} = \sqrt{mean(V_x^2 + V_y^2 + V_z^2)}
\]
- Indicates the total vibrational energy of the spindle–workpiece system.  
- Higher vibration levels correspond to chatter, imbalance, or worn-out cutting edges.  
- Used widely in condition monitoring for early detection of tool instability.

---

### **3. Mean Force–Vibration Angle (`mean_angle`)**
\[
\theta = \cos^{-1}\!\left(\frac{\vec{F}\cdot\vec{V}}{|\vec{F}||\vec{V}|}\right)
\]
- Captures the average alignment between force and vibration vectors.  
- When the cutting process is stable, these vectors are nearly aligned.  
- Deviations (larger angles) signify irregular contact, misalignment, or chatter.

---

### **4. Top-100 Angles Mean (`top100_angle`)**
- The average of the **100 largest instantaneous angles** observed during a cut.  
- Emphasizes rare but strong misalignment events, which often precede surface damage or catastrophic failure.  
- Provides sensitivity to intermittent chatter bursts that a simple mean would miss.

---

### **5. Acoustic Emission Mean (`AE_mean`)**
- Average amplitude of the AE-RMS signal.  
- AE signals reflect micro-fractures, friction, and plastic deformation.  
- As wear progresses, AE energy increases due to intensified material contact and micro-cracking.

---

### **6. Force Magnitude Variability (`F_std`)**
\[
F_{std} = std(F_{net})
\]
- Measures the fluctuation of cutting force magnitude during a cut.  
- Large variability indicates unstable chip formation or intermittent tool–workpiece engagement.  
- High `F_std` correlates with degraded cutting efficiency.

---

### **7. Vibration Kurtosis (`V_kurt`)**
- Statistical measure of “peakedness” of the vibration signal.  
- Sensitive to shocks or impulsive behavior, which occur when the tool edge chips or breaks.  
- High kurtosis is a classical indicator of mechanical impacts and faults.

---

### **8. Force–Vibration Correlation (`F_V_corr`)**
\[
corr(F_{net}, V_{net})
\]
- Quantifies dynamic coupling between the applied load and the structural response.  
- High correlation → predictable, stable process.  
- Decreasing correlation → nonlinear chatter or random impacts due to wear.

---

### **9. Force Spectral Centroid (`F_spec_centroid`)**
\[
C_f = \frac{\sum f_i \cdot |FFT(F_{net})_i|}{\sum |FFT(F_{net})_i|}
\]
- Represents the “center of mass” of the force signal spectrum.  
- A shift toward higher frequencies indicates chatter and vibration excitation at the spindle natural modes.  
- Effective for detecting transition from steady to unstable cutting regimes.

---

### **10. Acoustic Energy Ratio (`AE_energy_ratio`)**
\[
R_{AE} = \frac{E_{high}(10\text{–}20\,kHz)}{E_{low}(0\text{–}10\,kHz)}
\]
- Ratio of high-frequency to low-frequency AE energy components.  
- As the cutter dulls, more high-frequency bursts appear due to micro-fractures and abrasive wear.  
- Provides a frequency-domain indicator of tool health.

---

## 🧠 Summary

These 10 features are chosen to collectively capture:
- **Load behavior** → via force magnitude and variability  
- **Dynamic response** → via vibration energy and kurtosis  
- **Cross-interaction** → via angle and correlation measures  
- **Spectral indicators** → via force centroid and AE ratio  

Together, they condense each high-bandwidth cut into a compact, interpretable vector suitable for **RUL regression and wear-stage classification** models.


In [2]:
def extract_features(df, fs=50000):
    Fx, Fy, Fz = df.iloc[:,0], df.iloc[:,1], df.iloc[:,2]
    Vx, Vy, Vz = df.iloc[:,3], df.iloc[:,4], df.iloc[:,5]
    AE = df.iloc[:,6]
    
    # 1–2. Net RMS (force, vibration)
    F_net = np.sqrt(Fx**2 + Fy**2 + Fz**2)
    V_net = np.sqrt(Vx**2 + Vy**2 + Vz**2)
    F_rms = np.sqrt(np.mean(F_net**2))
    V_rms = np.sqrt(np.mean(V_net**2))
    
    # 3–4. Angle-based features
    cos_theta = np.clip((Fx*Vx + Fy*Vy + Fz*Vz) / 
                        (np.sqrt(Fx**2+Fy**2+Fz**2)*np.sqrt(Vx**2+Vy**2+Vz**2)+1e-8), -1, 1)
    theta = np.degrees(np.arccos(cos_theta))
    mean_angle = np.mean(theta)
    top100_angle = np.mean(np.sort(theta)[-100:]) if len(theta) >= 100 else np.mean(theta)
    
    # 5–7. Statistical
    AE_mean = np.mean(AE)
    F_std = np.std(F_net)
    V_kurt = kurtosis(V_net)
    
    # 8. Correlation
    corr = np.corrcoef(F_net, V_net)[0,1]
    
    # 9. Spectral centroid (force)
    yf = np.abs(rfft(F_net))
    xf = rfftfreq(len(F_net), 1/fs)
    spec_centroid = np.sum(xf * yf) / (np.sum(yf) + 1e-8)
    
    # 10. AE energy ratio
    ae_fft = np.abs(rfft(AE))
    ae_xf = rfftfreq(len(AE), 1/fs)
    low = np.sum(ae_fft[(ae_xf >= 0) & (ae_xf < 10000)])
    high = np.sum(ae_fft[(ae_xf >= 10000) & (ae_xf < 20000)])
    ae_energy_ratio = high / (low + 1e-8)
    
    return {
        'F_rms': F_rms, 'V_rms': V_rms,
        'mean_angle': mean_angle, 'top100_angle': top100_angle,
        'AE_mean': AE_mean, 'F_std': F_std, 'V_kurt': V_kurt,
        'F_V_corr': corr, 'F_spec_centroid': spec_centroid,
        'AE_energy_ratio': ae_energy_ratio
    }


In [3]:
RAW_DIR = "c1"
OUT_DIR = "preprocess"
os.makedirs(OUT_DIR, exist_ok=True)

files = sorted([f for f in os.listdir(RAW_DIR) if f.endswith(".csv")])

for f in files:
    raw_path = os.path.join(RAW_DIR, f)
    out_path = os.path.join(OUT_DIR, f)
    
    df = pd.read_csv(raw_path, header=None)
    feats = extract_features(df)
    pd.DataFrame([feats]).to_csv(out_path, index=False)
    
    print(f"Processed {f} → {out_path}")


Processed c_1_001.csv → preprocess\c_1_001.csv
Processed c_1_002.csv → preprocess\c_1_002.csv
Processed c_1_003.csv → preprocess\c_1_003.csv
Processed c_1_004.csv → preprocess\c_1_004.csv
Processed c_1_005.csv → preprocess\c_1_005.csv
Processed c_1_006.csv → preprocess\c_1_006.csv
Processed c_1_007.csv → preprocess\c_1_007.csv
Processed c_1_008.csv → preprocess\c_1_008.csv
Processed c_1_009.csv → preprocess\c_1_009.csv
Processed c_1_010.csv → preprocess\c_1_010.csv
Processed c_1_011.csv → preprocess\c_1_011.csv
Processed c_1_012.csv → preprocess\c_1_012.csv
Processed c_1_013.csv → preprocess\c_1_013.csv
Processed c_1_014.csv → preprocess\c_1_014.csv
Processed c_1_015.csv → preprocess\c_1_015.csv
Processed c_1_016.csv → preprocess\c_1_016.csv
Processed c_1_017.csv → preprocess\c_1_017.csv
Processed c_1_018.csv → preprocess\c_1_018.csv
Processed c_1_019.csv → preprocess\c_1_019.csv
Processed c_1_020.csv → preprocess\c_1_020.csv
Processed c_1_021.csv → preprocess\c_1_021.csv
Processed c_1